In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold

In [6]:
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Reemplaza los siguientes directorios por los tuyos
train_dirs = ['/Users/baudi/AI/practicas/uvas/data/train_val/']
batch_size = 32
img_height = 150
img_width = 150
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Función para aplicar data augmentation a un lote de imágenes
def data_augmentation(images, labels):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomWidth(0.2),
        tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    ])
    return data_augmentation(images), labels

# Carga y aumenta las imágenes utilizando tf.data y prefetch
train_ds_list = [image_dataset_from_directory(
    directory,
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size) for directory in train_dirs]

train_ds = train_ds_list[0]
for ds in train_ds_list[1:]:
    train_ds = train_ds.concatenate(ds)

train_ds = train_ds.shuffle(1000).map(data_augmentation, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE)

# Construye y compila tu CNN aquí
model = build_model()
# Entrena tu CNN usando la data augmentation
model.fit(
    train_ds,
    epochs=20
)


Found 3248 files belonging to 4 classes.
Epoch 1/20


2023-04-08 16:06:04.066873: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


102/102 [==============================] - 6s 45ms/step - loss: 0.7332 - accuracy: 0.7669
Epoch 2/20
102/102 [==============================] - 5s 43ms/step - loss: 0.2769 - accuracy: 0.9033
Epoch 3/20
102/102 [==============================] - 5s 43ms/step - loss: 0.1488 - accuracy: 0.9504
Epoch 4/20
102/102 [==============================] - 5s 44ms/step - loss: 0.1244 - accuracy: 0.9603
Epoch 5/20
102/102 [==============================] - 5s 44ms/step - loss: 0.0767 - accuracy: 0.9729
Epoch 6/20
102/102 [==============================] - 5s 43ms/step - loss: 0.0536 - accuracy: 0.9809
Epoch 7/20
102/102 [==============================] - 5s 42ms/step - loss: 0.0467 - accuracy: 0.9846
Epoch 8/20
102/102 [==============================] - 5s 42ms/step - loss: 0.0424 - accuracy: 0.9840
Epoch 9/20
102/102 [==============================] - 5s 44ms/step - loss: 0.0326 - accuracy: 0.9883
Epoch 10/20
102/102 [==============================] - 5s 47ms/step - loss: 0.0456 - accuracy: 0.9828


In [3]:
def build_model():

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 clases: sanas, enfermedad1, enfermedad2, enfermedad3
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


In [9]:
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/'

# Crear un generador de imágenes para el conjunto de pruebas sin Data Augmentation
test_datagen = ImageDataGenerator(
    rescale=1./255,
)
test_data = test_datagen.flow_from_directory(
    test_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# Evaluar el modelo final en los datos de prueba
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Found 814 images belonging to 4 classes.
26/26 [==============================] - 1s 20ms/step - loss: 11.2825 - accuracy: 0.2899
Test Loss: 11.2825
Test Accuracy: 0.2899
